In [1]:
import pandas as pd
import numpy as np

import json

import torch
import torch.nn as nn
from torch.nn import functional as F

from PIL import Image
from transformers import CLIPTokenizer

from tqdm import tqdm

In [2]:
df = pd.read_csv('dataset/marking.csv', index_col='image_id')
df['text'] = df['text'].str.lower()
df['text_length'] = df['text'].apply(len)
df['word_count'] = df['text'].apply(lambda x: len(x.split()))
df.head()

,text,name,group,sub_group,emoji,train,text_length,word_count
image_id,,,,,,,,
0,ухмыляющееся лицо,grinning face,Smileys & Emotion,face-smiling,😀,1,17,2
1,ухмыляющееся лицо с большими глазами,smiling face with open mouth,Smileys & Emotion,face-smiling,😃,1,36,5
2,улыбающееся лицо с улыбающимися глазами,smiling face with open mouth and smiling eyes,Smileys & Emotion,face-smiling,😄,1,39,5
3,сияющее лицо с улыбающимися глазами,grinning face with smiling eyes,Smileys & Emotion,face-smiling,😁,1,35,5
4,ухмыляющееся щурящееся лицо,smiling face with open mouth and tightly close...,Smileys & Emotion,face-smiling,😆,1,27,3


In [3]:
df.columns

Index(['text', 'name', 'group', 'sub_group', 'emoji', 'train', 'text_length',
       'word_count'],
      dtype='object')

In [4]:
## Length of df
len(df)

2749

In [8]:
# Filter rows where 'train' column is True if you only want those marked for training
# df_train = df[df['train'] == True]

# Define the output file for the metadata
metadata_jsonl_path = 'dataset/rgba_train/metadata.jsonl'

# Open the output file
with open(metadata_jsonl_path, 'w') as f:
    # Iterate over each row in the dataframe
    for index, row in df.iterrows():
        # Construct the metadata entry
        metadata_entry = {
            "file_name": f"train_{index:05d}.png",  # This assumes the index matches the image numbering
            "text": row['name']  # Change to row['text'] if you prefer the Russian description
        }

        # Write the metadata entry to the file
        f.write(json.dumps(metadata_entry) + '\n')

print(f"Metadata.jsonl file created at: {metadata_jsonl_path}")

Metadata.jsonl file created at: dataset/rgba_train/metadata.jsonl


In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git
!pip install -U -r requirements.txt

In [10]:
!accelerate config

/home/fiwex/.pyenv/versions/InstaFlow/lib/python3.11/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
--------------------------------------------------------------------------------In which compute environment are you running?
Please select a choice using the arrow or number keys, and selecting with enter
 ➔  This machine
    AWS (Amazon SageMaker)
Traceback (most recent call last):
  File "/home/fiwex/.pyenv/versions/3.11.7/envs/InstaFlow/bin/accelerate", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/home/fiwex/.pyenv/versions/InstaFlow/lib/python3.11/site-packages/accelerate/commands/accelerate_cli.py", line 47, in main
    args.func(args)
  File "/home/fiwex/.pyenv/versions/InstaFlow/lib/python3.11/site-packages/accelerate/commands/config/config.py", line 67, in config_command
    conf

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 

In [ ]:
!export MODEL_NAME="runwayml/stable-diffusion-v1-5"

In [ ]:
!export TRAIN_DIR="dataset/rgba_train"

In [ ]:
!export OUTPUT_DIR="stable_diffusion_files"

In [ ]:
%%bash

accelerate launch train_text_to_image.py \
 --pretrained_model_name_or_path={os.environ['MODEL_NAME']} \
 --train_data_dir={os.environ['TRAIN_DIR']} \
 --use_ema \
                                                 --resolution=512 --center_crop --random_flip \
                                                              --train_batch_size=1 \
                                                                                 --gradient_accumulation_steps=4 \
                                                                                                               --gradient_checkpointing \
                                                                                                               --mixed_precision="fp16" \
                                                                                                                                 --max_train_steps=15000 \
                                                                                                                                                   --learning_rate=1e-05 \
                                                                                                                                                                   --max_grad_norm=1 \
                                                                                                                                                                                   --lr_scheduler="constant" --lr_warmup_steps=0 \
                                                                                                                                                                                                                               --output_dir={os.environ['OUTPUT_DIR']}